In [1]:
%%javascript
require.config({paths: {
        vis: "http://cdnjs.cloudflare.com/ajax/libs/vis/4.17.0/vis",
        yadage: "https://rawgit.com/lukasheinrich/yadage-ipython/master/yadage"
    }
});

<IPython.core.display.Javascript object>

In [2]:
import shutil
import os
import logging

import yadage_widget
import yadage.steering_object
from yadage.clihelpers import setupbackend_fromstring, prepare_workdir_from_archive

## prepare workflow

In [3]:
repolocation = 'https://raw.githubusercontent.com/lukasheinrich/weinberg-exp/master/example_yadage'
workdir = '/srv/workerdata/ipythonwork'
try:
    shutil.rmtree(workdir)
except OSError:
    pass
finally:
    prepare_workdir_from_archive(
        workdir,
        '{}/input.zip'.format(repolocation)
    )

In [4]:
ys = yadage.steering_object.YadageSteering()
ys.prepare_workdir(workdir, accept_existing_workdir = True)

initdata = {
        'nevents':10000,
        'seeds':[1,2,3],
        'runcardtempl':'run_card.templ',
        'proccardtempl':'sm_proc_card.templ',
        'paramcardtempl':'param_card.templ',
        'sqrtshalf':45,
        'polbeam1':0,
        'polbeam2':0,
        'Gf': 1.766390e-05
}
ys.init_workflow(
    'rootflow.yml',
    'github:lukasheinrich/weinberg-exp:example_yadage',
    initdata, initdir = '{}/workdir/init'.format(workdir)
)


In [5]:
ui = yadage_widget.WorkflowWidget(ys.controller.adageobj)
ys.adage_argument(additional_trackers = [ui.adagetracker])
ui

## Execute Workflow

In [6]:
backend = setupbackend_fromstring('celery')
ys.run_adage(backend)